# Community FBA models (TODO)

In [1]:
using COBREXA

Here we will construct a community FBA model of two  *E. coli* "core" models
that can interact by exchanging selected metabolites. To do this, we will need
the model, which we can download if it is not already present.

In [2]:
download_model(
    "http://bigg.ucsd.edu/static/models/e_coli_core.json",
    "e_coli_core.json",
    "7bedec10576cfe935b19218dc881f3fb14f890a1871448fc19a9b4ee15b448d8",
)

[ Info: using cached `e_coli_core.json'


"e_coli_core.json"

Additionally to COBREXA and the model format package, we will need a solver
-- let's use GLPK here:

In [3]:
import JSONFBCModels
import GLPK
import AbstractFBCModels.CanonicalModel as CM
import ConstraintTrees as C

ecoli1 = load_model("e_coli_core.json", CM.Model)
ecoli1.reactions["EX_glc__D_e"].lower_bound = -1000.0
ecoli1.reactions["EX_glc__D_e"].upper_bound = 1000.0
ecoli2 = deepcopy(ecoli1)

AbstractFBCModels.CanonicalModel.Model(
  reactions = Dict{String, AbstractFBCModels.CanonicalModel.Reaction}("ACALD" =…
  metabolites = Dict{String, AbstractFBCModels.CanonicalModel.Metabolite}("co2_…
  genes = Dict{String, AbstractFBCModels.CanonicalModel.Gene}("b4301" => Abstra…
)


customize models a bit

In [4]:
ecoli1.reactions["CYTBD"].lower_bound = ecoli1.reactions["CYTBD"].upper_bound = 0.0
ecoli2.reactions["FBA"].lower_bound = ecoli2.reactions["FBA"].upper_bound = 0.0

0.0

## Analysing the community

In [5]:
my_community = Dict("bug1" => (ecoli1, 0.2), "bug2" => (ecoli2, 0.8))

solution = community_flux_balance_analysis(
    my_community,
    ["EX_glc__D_e" => (-10.0, 0.0)],
    optimizer = GLPK.Optimizer,
)

ConstraintTrees.Tree{Float64} with 6 elements:
  :bug1                => ConstraintTrees.Tree{Float64}(#= 4 elements =#)
  :bug2                => ConstraintTrees.Tree{Float64}(#= 4 elements =#)
  :community_balance   => ConstraintTrees.Tree{Float64}(#= 20 elements =#)
  :community_biomass   => 0.523716
  :community_exchanges => ConstraintTrees.Tree{Float64}(#= 20 elements =#)
  :equal_growth        => ConstraintTrees.Tree{Float64}(#= 1 element =#)

## Investigating the solution

We can now e.g. observe the differences in individual pairs of exchanges:

In [6]:
C.zip(
    tuple,
    solution.bug1.interface.exchanges,
    solution.bug2.interface.exchanges,
    Tuple{Float64,Float64},
)

ConstraintTrees.Tree{Tuple{Float64, Float64}} with 20 elements:
  :EX_ac_e     => (16.5424, 1.06581e-14)
  :EX_acald_e  => (0.0, 0.0)
  :EX_akg_e    => (0.0, 0.0)
  :EX_co2_e    => (-0.935723, 19.8177)
  :EX_etoh_e   => (15.9879, 0.0)
  :EX_for_e    => (35.0581, 2.52782)
  :EX_fru_e    => (0.0, 0.0)
  :EX_fum_e    => (0.0, 0.0)
  :EX_glc__D_e => (-20.245, -7.43875)
  :EX_gln__L_e => (0.0, 0.0)
  :EX_glu__L_e => (4.88498e-15, 0.0)
  :EX_h2o_e    => (-13.1086, 23.6327)
  :EX_h_e      => (62.1062, 13.0336)
  :EX_lac__D_e => (7.10543e-15, 0.0)
  :EX_mal__L_e => (0.0, 0.0)
  :EX_nh4_e    => (-2.85572, -2.85572)
  :EX_o2_e     => (0.0, -20.4762)
  :EX_pi_e     => (-1.92659, -1.92659)
  :EX_pyr_e    => (0.0, 0.0)
  :EX_succ_e   => (3.55271e-15, 0.0)

...or use `screen` to efficiently find out which composition is best:

In [7]:
screen(0.0:0.1:1.0) do ratio2
    ratio1 = 1 - ratio2
    res = community_flux_balance_analysis(
        [("bug1" => (ecoli1, ratio1)), ("bug2" => (ecoli2, ratio2))],
        ["EX_glc__D_e" => (-10.0, 0.0)],
        interface = :sbo, # usually more reproducible
        optimizer = GLPK.Optimizer,
    )
    (ratio1, ratio2) => (isnothing(res) ? nothing : res.community_biomass)
end

11-element Vector{Pair{Tuple{Float64, Float64}, Float64}}:
                 (1.0, 0.0) => 0.21166294973531166
                 (0.9, 0.1) => 0.23424604629802845
                 (0.8, 0.2) => 0.25971971809640987
                 (0.7, 0.3) => 0.2886768902449847
                 (0.6, 0.4) => 0.32188455339556993
                 (0.5, 0.5) => 0.36035269877962506
                 (0.4, 0.6) => 0.4054388316161014
 (0.30000000000000004, 0.7) => 0.4590115020201965
 (0.19999999999999996, 0.8) => 0.5237157737585179
 (0.09999999999999998, 0.9) => 0.6034233598466001
                 (0.0, 1.0) => 0.7040369478590228

...seems a lot like `bug1` will eventually disappear.

## Inspecting the interfaces

Not all interfaces are made equally! Fortunately, it is simple to create a
custom interface, by just manually assigning reactions to semantic groups
using ConstraintTrees.

Some work:

In [8]:
flux_balance_constraints(ecoli1, interface = :sbo).interface

ConstraintTrees.Tree{ConstraintTrees.Constraint} with 2 elements:
  :biomass   => ConstraintTrees.Tree{ConstraintTrees.Constraint}(#= 1 element =…
  :exchanges => ConstraintTrees.Tree{ConstraintTrees.Constraint}(#= 20 elements…

Some generally don't do well:

In [9]:
flux_balance_constraints(ecoli1, interface = :boundary).interface

ConstraintTrees.Tree{ConstraintTrees.Constraint} with 1 element:
  :boundary => ConstraintTrees.Tree{ConstraintTrees.Constraint}(#= 20 elements …

Do it manually:

In [10]:
own_interface = deepcopy(flux_balance_constraints(ecoli1))
own_interface *=
    :interface^C.ConstraintTree(
        :biomass => own_interface.fluxes.BIOMASS_Ecoli_core_w_GAM,
        :exchanges => C.ConstraintTree(
            k => v for (k, v) in own_interface.fluxes if startswith(string(k), "EX_")
        ),
    )
own_interface.interface.exchanges

ConstraintTrees.Tree{ConstraintTrees.Constraint} with 20 elements:
  :EX_ac_e     => ConstraintTrees.Constraint(ConstraintTrees.LinearValue(#= ...…
  :EX_acald_e  => ConstraintTrees.Constraint(ConstraintTrees.LinearValue(#= ...…
  :EX_akg_e    => ConstraintTrees.Constraint(ConstraintTrees.LinearValue(#= ...…
  :EX_co2_e    => ConstraintTrees.Constraint(ConstraintTrees.LinearValue(#= ...…
  :EX_etoh_e   => ConstraintTrees.Constraint(ConstraintTrees.LinearValue(#= ...…
  :EX_for_e    => ConstraintTrees.Constraint(ConstraintTrees.LinearValue(#= ...…
  :EX_fru_e    => ConstraintTrees.Constraint(ConstraintTrees.LinearValue(#= ...…
  :EX_fum_e    => ConstraintTrees.Constraint(ConstraintTrees.LinearValue(#= ...…
  :EX_glc__D_e => ConstraintTrees.Constraint(ConstraintTrees.LinearValue(#= ...…
  :EX_gln__L_e => ConstraintTrees.Constraint(ConstraintTrees.LinearValue(#= ...…
  :EX_glu__L_e => ConstraintTrees.Constraint(ConstraintTrees.LinearValue(#= ...…
  :EX_h2o_e    => ConstraintTrees.Constrai

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*